In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [32]:
df = pd.read_excel(r"D:\Pharma_Inventory_Optimization_Project\03_Model\Safety_Stock_Model.xlsx")

In [33]:
df.head()

,Item,Average_Demand,StdDev_Demand,Lead_Time
0,1,21.981599,8.468922,7
1,2,58.574151,20.093015,7
2,3,36.642223,13.179441,7
3,4,22.010241,8.403898,7
4,5,18.358708,7.265167,7


In [34]:
df.columns

Index(['Item', 'Average_Demand', 'StdDev_Demand', 'Lead_Time'], dtype='str')

In [35]:
df["Service_Level"] = 0.95

#i cannot calculate the ACV annual consumption value as i dont have the cost in the dataset

In [ ]:
from scipy.stats import norm

df["Z_value"] = norm.ppf(df["Service_Level"]).round(2)

#if i change service level percentage then this Z value will also change

In [37]:
df.head()

,Item,Average_Demand,StdDev_Demand,Lead_Time,Service_Level,Z_value
0,1,21.981599,8.468922,7,0.95,1.64
1,2,58.574151,20.093015,7,0.95,1.64
2,3,36.642223,13.179441,7,0.95,1.64
3,4,22.010241,8.403898,7,0.95,1.64
4,5,18.358708,7.265167,7,0.95,1.64


In [38]:
df.rename(columns={"StdDev_Demand": "Std_Demand"}, inplace=True)
df.rename(columns={"Average_Demand": "Avg_Demand"}, inplace=True)

In [39]:
df["Avg_Demand"] = round(df["Avg_Demand"],2)
df["Std_Demand"] = round(df["Std_Demand"],2)

In [40]:
df.head()

,Item,Avg_Demand,Std_Demand,Lead_Time,Service_Level,Z_value
0,1,21.98,8.47,7,0.95,1.64
1,2,58.57,20.09,7,0.95,1.64
2,3,36.64,13.18,7,0.95,1.64
3,4,22.01,8.40,7,0.95,1.64
4,5,18.36,7.27,7,0.95,1.64


In [41]:
df["Safety_Stock"] = round(df["Std_Demand"] * df["Z_value"] * np.sqrt(df["Lead_Time"]),0)
#square root of lead time
# I did round of till 0 cause the safety stock value can never be in points

In [42]:
df["Reorder_Point"] = round((df["Avg_Demand"] * df["Lead_Time"]) + df["Safety_Stock"],0)

In [43]:
df["CV_%"] = round((df["Std_Demand"] / df["Avg_Demand"]) * 100,0)

In [44]:
df.tail()

,Item,Avg_Demand,Std_Demand,Lead_Time,Service_Level,Z_value,Safety_Stock,Reorder_Point,CV_%
45,46,58.64,20.22,3,0.95,1.64,57.0,233.0,34.0
46,47,22.00,8.42,3,0.95,1.64,24.0,90.0,38.0
47,48,51.35,17.88,3,0.95,1.64,51.0,205.0,35.0
48,49,29.34,10.87,3,0.95,1.64,31.0,119.0,37.0
49,50,65.88,22.42,3,0.95,1.64,64.0,262.0,34.0


In [45]:
df.nlargest(1,"CV_%")

,Item,Avg_Demand,Std_Demand,Lead_Time,Service_Level,Z_value,Safety_Stock,Reorder_Point,CV_%
4,5,18.36,7.27,7,0.95,1.64,32.0,161.0,40.0


In [46]:
df["CV_%"].count()

np.int64(50)

In [47]:
df = df.sort_values(by="Avg_Demand", ascending=False).reset_index(drop=True)

In [48]:
drug_names = [
"Paracetamol_500mg","Ibuprofen_400mg","Metformin_500mg","Amoxicillin_250mg",
"Azithromycin_500mg","Amlodipine_5mg","Pantoprazole_40mg","Aspirin_75mg",
"Telmisartan_40mg","Atorvastatin_10mg",
"Cefixime_200mg","Ciprofloxacin_500mg","Diclofenac_50mg","Glimepiride_2mg",
"Losartan_50mg","Rosuvastatin_20mg","Omeprazole_20mg","Levocetirizine_5mg",
"Montelukast_10mg","Sitagliptin_100mg",
"Doxycycline_100mg","Clopidogrel_75mg","Gabapentin_300mg","Tramadol_50mg",
"Sertraline_50mg","Escitalopram_10mg","Furosemide_40mg","Spironolactone_25mg",
"Metoprolol_50mg","Propranolol_40mg",
"Vitamin_D3_60000IU","Iron_Folic_Acid","Calcium_Carbonate","Ketoconazole_Cream",
"Mupirocin_Ointment","Loperamide_2mg","Ondansetron_4mg","Warfarin_5mg",
"Heparin_Injection","Cefuroxime_500mg","Amikacin_Injection",
"Prednisolone_10mg","Insulin_Glargine","Salbutamol_Inhaler",
"Budesonide_Inhaler","Hydroxychloroquine_200mg","Ranitidine_150mg","Enalapril_5mg",
"Clarithromycin_500mg","Rivaroxaban_10mg"
]
#The raw dataset had item IDs. I mapped them to product names using a reference list for better interpretability and dashboard presentation.

In [49]:
df["Item_Name"] = drug_names[:len(df)]


In [50]:
df = df.sort_values(by="Item").reset_index(drop=True)

In [51]:
df.head()

,Item,Avg_Demand,Std_Demand,Lead_Time,Service_Level,Z_value,Safety_Stock,Reorder_Point,CV_%,Item_Name
0,1,21.98,8.47,7,0.95,1.64,37.0,191.0,39.0,Clarithromycin_500mg
1,2,58.57,20.09,7,0.95,1.64,87.0,497.0,34.0,Clopidogrel_75mg
2,3,36.64,13.18,7,0.95,1.64,57.0,313.0,36.0,Loperamide_2mg
3,4,22.01,8.40,7,0.95,1.64,36.0,190.0,38.0,Hydroxychloroquine_200mg
4,5,18.36,7.27,7,0.95,1.64,32.0,161.0,40.0,Rivaroxaban_10mg


In [52]:
df.isnull().value_counts()

Item   Avg_Demand  Std_Demand  Lead_Time  Service_Level  Z_value  Safety_Stock  Reorder_Point  CV_%   Item_Name
False  False       False       False      False          False    False         False          False  False        50
Name: count, dtype: int64

In [53]:
df.head()

,Item,Avg_Demand,Std_Demand,Lead_Time,Service_Level,Z_value,Safety_Stock,Reorder_Point,CV_%,Item_Name
0,1,21.98,8.47,7,0.95,1.64,37.0,191.0,39.0,Clarithromycin_500mg
1,2,58.57,20.09,7,0.95,1.64,87.0,497.0,34.0,Clopidogrel_75mg
2,3,36.64,13.18,7,0.95,1.64,57.0,313.0,36.0,Loperamide_2mg
3,4,22.01,8.40,7,0.95,1.64,36.0,190.0,38.0,Hydroxychloroquine_200mg
4,5,18.36,7.27,7,0.95,1.64,32.0,161.0,40.0,Rivaroxaban_10mg


In [54]:
category_mapping = {

# Pain Relief
"Paracetamol_500mg": "Pain Relief",
"Ibuprofen_400mg": "Pain Relief",
"Diclofenac_50mg": "Pain Relief",
"Tramadol_50mg": "Pain Relief",

# Antibiotics
"Amoxicillin_250mg": "Antibiotic",
"Azithromycin_500mg": "Antibiotic",
"Cefixime_200mg": "Antibiotic",
"Ciprofloxacin_500mg": "Antibiotic",
"Doxycycline_100mg": "Antibiotic",
"Cefuroxime_500mg": "Antibiotic",
"Amikacin_Injection": "Antibiotic",
"Clarithromycin_500mg": "Antibiotic",

# Diabetes
"Metformin_500mg": "Diabetes",
"Glimepiride_2mg": "Diabetes",
"Sitagliptin_100mg": "Diabetes",
"Insulin_Glargine": "Diabetes",

# Cardiovascular
"Amlodipine_5mg": "Cardiovascular",
"Telmisartan_40mg": "Cardiovascular",
"Losartan_50mg": "Cardiovascular",
"Atorvastatin_10mg": "Cardiovascular",
"Rosuvastatin_20mg": "Cardiovascular",
"Metoprolol_50mg": "Cardiovascular",
"Propranolol_40mg": "Cardiovascular",
"Warfarin_5mg": "Cardiovascular",
"Heparin_Injection": "Cardiovascular",
"Clopidogrel_75mg": "Cardiovascular",
"Spironolactone_25mg": "Cardiovascular",
"Furosemide_40mg": "Cardiovascular",
"Enalapril_5mg": "Cardiovascular",
"Rivaroxaban_10mg": "Cardiovascular",
"Aspirin_75mg": "Cardiovascular",

# Gastro
"Pantoprazole_40mg": "Gastro",
"Omeprazole_20mg": "Gastro",
"Ranitidine_150mg": "Gastro",
"Loperamide_2mg": "Gastro",
"Ondansetron_4mg": "Gastro",

# Allergy
"Levocetirizine_5mg": "Allergy",
"Montelukast_10mg": "Allergy",

# Neurology / Psychiatric
"Gabapentin_300mg": "Neurology",
"Sertraline_50mg": "Psychiatric",
"Escitalopram_10mg": "Psychiatric",

# Supplements
"Vitamin_D3_60000IU": "Supplement",
"Iron_Folic_Acid": "Supplement",
"Calcium_Carbonate": "Supplement",

# Respiratory
"Salbutamol_Inhaler": "Respiratory",
"Budesonide_Inhaler": "Respiratory",

# Steroid / Immunology
"Prednisolone_10mg": "Steroid",
"Hydroxychloroquine_200mg": "Immunology",

# Dermatology
"Ketoconazole_Cream": "Dermatology",
"Mupirocin_Ointment": "Dermatology"
}

df["Category"] = df["Item_Name"].map(category_mapping)

In [55]:
df = df[[
    "Item",
    "Item_Name",
    "Category",
    "Avg_Demand",
    "Std_Demand",
    "CV_%",
    "Lead_Time",
    "Service_Level",
    "Z_value",
    "Safety_Stock",
    "Reorder_Point"
]]

In [56]:
df.head()

,Item,Item_Name,Category,Avg_Demand,Std_Demand,CV_%,Lead_Time,Service_Level,Z_value,Safety_Stock,Reorder_Point
0,1,Clarithromycin_500mg,Antibiotic,21.98,8.47,39.0,7,0.95,1.64,37.0,191.0
1,2,Clopidogrel_75mg,Cardiovascular,58.57,20.09,34.0,7,0.95,1.64,87.0,497.0
2,3,Loperamide_2mg,Gastro,36.64,13.18,36.0,7,0.95,1.64,57.0,313.0
3,4,Hydroxychloroquine_200mg,Immunology,22.01,8.40,38.0,7,0.95,1.64,36.0,190.0
4,5,Rivaroxaban_10mg,Cardiovascular,18.36,7.27,40.0,7,0.95,1.64,32.0,161.0


In [58]:
df.to_csv("Safety_Stock.csv")